In [17]:
import pandas as pd

def clean_and_load_csv(file_path):
    """Read a CSV file, strip whitespace from column names, and return the DataFrame."""
    df = pd.read_csv(file_path)  # Load the CSV file into a DataFrame
    df.columns = df.columns.str.strip()  # Strip whitespace from column names
    return df  # Return the cleaned DataFrame

def filter_gsv(df1, df2, threshold=10000):
    """Filter two DataFrames based on 'gsv' column."""
    for df in [df1, df2]:
        # Check if 'gsv' column exists in both DataFrames
        if 'gsv' not in df.columns:
            raise KeyError("The DataFrame does not contain a 'gsv' column. Available columns are: " + ", ".join(df.columns))

    # Convert 'gsv' to numeric and filter based on the threshold for df1
    df1.loc[:, 'gsv'] = pd.to_numeric(df1['gsv'], errors='coerce')  # Convert 'gsv' to numeric, coercing errors to NaN
    df1 = df1.dropna(subset=['gsv'])  # Drop rows where 'gsv' is NaN
    filtered_df1 = df1[df1['gsv'] >= threshold]  # Filter df1 for 'gsv' values greater than or equal to the threshold
    
    # Convert 'gsv' to numeric and filter based on the threshold for df2
    df2.loc[:, 'gsv'] = pd.to_numeric(df2['gsv'], errors='coerce')  # Convert 'gsv' to numeric, coercing errors to NaN
    df2 = df2.dropna(subset=['gsv'])  # Drop rows where 'gsv' is NaN
    filtered_df2 = df2[df2['gsv'] >= threshold]  # Filter df2 for 'gsv' values greater than or equal to the threshold

    return filtered_df1, filtered_df2  # Return the filtered DataFrames

def filter_and_display(df5, source_partner):
    """Filter df5 to get top 10 rows for each source country based on 'gsv'."""
    # Check if required columns exist in df5
    if 'source_country' not in df5.columns or 'destination_country' not in df5.columns or 'gsv' not in df5.columns:
        raise KeyError("One of the required columns 'source_country', 'destination_country', or 'gsv' does not exist.")
    
    # Skip rows where source_country is equal to destination_country
    df5 = df5[df5['source_country'] != df5['destination_country']]
    
    # Add source_partner to the DataFrame
    df5['source_partner'] = source_partner  # Correct assignment for the current partner
    
    # Sort by gsv in descending order and get top 10 for each source country
    top_rows = df5.sort_values(by=['source_country', 'gsv'], ascending=[False, False]).groupby('source_country').head(10)

    return top_rows  # Return the top rows for each source country

def compare_and_filter(top_rows, filtered_df1):
    """Compare top_rows with filtered_df1 and keep only matching rows."""
    # Check if the required columns exist in filtered_df1
    if 'source_partner' not in filtered_df1.columns or 'source_country' not in filtered_df1.columns:
        raise KeyError("filtered_df1 must contain 'source_partner' and 'source_country' columns.")

    # Create a set of tuples from filtered_df1 for quick lookup
    filtered_set = set(zip(filtered_df1['source_partner'], filtered_df1['source_country']))

    # Filter top_rows to keep only those that exist in filtered_df1
    top_rows_filtered = top_rows[top_rows.apply(lambda x: (x['source_partner'], x['source_country']) in filtered_set, axis=1)]

    return top_rows_filtered  # Return the filtered top rows

def merge_duplicate_rows(final_results):
    """Merge duplicate rows based on source_partner, source_country, and destination_country."""
    # Group by the specified columns and sum the gsv values
    merged_results = final_results.groupby(
        ['source_partner', 'source_country', 'destination_country'], as_index=False
    ).agg({'gsv': 'sum'})  # Aggregate gsv by summing duplicates

    return merged_results  # Return the merged results

def add_regional_director(merged_results, r_d_file_path):
    """Add a 'regional_director' column to merged_results based on matches in R_D.csv."""
    # Load the R_D.csv file
    r_d_df = pd.read_csv(r_d_file_path)
    
    # Strip whitespace from column names in R_D DataFrame
    r_d_df.columns = r_d_df.columns.str.strip()
    
    # Check if 'source_partner' and 'regional_director' columns exist in R_D DataFrame
    if 'source_partner' not in r_d_df.columns or 'regional_director' not in r_d_df.columns:
        raise KeyError("R_D.csv must contain 'source_partner' and 'regional_director' columns.")
    
    # Merge merged_results with r_d_df on 'source_partner'
    merged_with_director = pd.merge(
        merged_results,
        r_d_df[['source_partner', 'regional_director']],  # Select only the necessary columns
        on='source_partner',
        how='left'  # Use left join to keep all rows from merged_results
    )
    
    return merged_with_director  # Return the merged DataFrame with regional director
    
def add_partner_gsv(final_results, df):
    """Add a 'Partner_Gsv' column to final_results based on matches in the original DataFrame."""
    # Initialize the Partner_Gsv column with NaN
    final_results['Partner_Gsv'] = pd.NA

    # Iterate through each row in final_results
    for index, row in final_results.iterrows():
        # Filter the original DataFrame for matching source_partner, source_country, and destination_country
        matching_rows = df[
            (df['source_partner'] == row['source_partner']) &
            (df['source_country'] == row['source_country']) &
            (df['destination_country'] == row['destination_country'])
        ]
        
        # If there are matching rows, sum their gsv values and assign to Partner_Gsv
        if not matching_rows.empty:
            final_results.at[index, 'Partner_Gsv'] = matching_rows['gsv'].sum()

    # Add the 'existing/new' column based on the 'Partner_Gsv' values
    final_results['existing/new'] = final_results['Partner_Gsv'].apply(lambda x: 'EXISTING' if pd.notna(x) else 'NEW')

    # Reorder columns to place Partner_Gsv and existing/new after gsv
    cols = list(final_results.columns)
    gsv_index = cols.index('gsv')
    cols.insert(gsv_index + 1, cols.pop(cols.index('Partner_Gsv')))
    cols.insert(gsv_index + 2, cols.pop(cols.index('existing/new')))
    final_results = final_results[cols]

    return final_results  # Return the updated final results with Partner_Gsv and existing/new

def rename_gsv_column(final_results):
    """Rename the 'gsv' column to 'TerraPay_gsv' in the final results."""
    # Check if 'gsv' column exists
    if 'gsv' in final_results.columns:
        final_results = final_results.rename(columns={'gsv': 'TerraPay_gsv'})  # Rename the column
    else:
        raise KeyError("'gsv' column not found in the DataFrame.")  # Raise an error if 'gsv' column doesn't exist

    return final_results  # Return the DataFrame with the renamed column

def process_csv(final_results):
    """
    Reads a CSV file, sorts by gsv in descending order,
    and displays all columns while grouping by source_partner and source_country.

    Parameters:
    - file_name: str - Name of the input CSV file (in the Jupyter directory).
    """

    # Clean column names by removing leading and trailing whitespace
    df.columns = df.columns.str.strip()

    # Sort the DataFrame by gsv in descending order
    sorted_df = df.sort_values(by=['source_partner', 'source_country', 'gsv'], ascending=[True, True, False])

    # Set display options to show all columns
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent line breaks in the output

    # Display the sorted DataFrame
    print(sorted_df)

    # Optionally, save the sorted DataFrame to a new CSV file
    output_file = 'partner_opportunities.csv'
    sorted_df.to_csv(output_file, index=False)
    print(f"\nSorted data saved to '{output_file}'")

process_csv(file_name)

if __name__ == "__main__":
    # Clean and load the first DataFrame
    df = clean_and_load_csv('P_B.csv')  # Assuming this contains source_partner
    print("Columns in df:", df.columns.tolist())  # Display the columns in the first DataFrame
    
    # Clean and load the second DataFrame
    df2 = clean_and_load_csv('TP_B.csv')  # Assuming this contains source_country and gsv
    print("Columns in df2:", df2.columns.tolist())  # Display the columns in the second DataFrame
    
    # Filter the DataFrames based on 'gsv'
    filtered_df1, filtered_df2 = filter_gsv(df, df2)  # Get filtered DataFrames based on gsv
    
    # Display the filtered DataFrames
    print("Filtered DataFrame 1:\n", filtered_df1)  # Show filtered DataFrame 1
    print("Filtered DataFrame 2:\n", filtered_df2)  # Show filtered DataFrame 2

    
   # Example list of source partners
    source_partners = ['33 Remittance Company Limited - smartswift', '9PAY JOINT STOCK COMPANY', 'Abu Sheika Exchange', 'Access Forex Proprietary Ltd', 'Afex transferencias y cambios Ltda', 
  'Afmoom Services Pty Ltd', 'Afriex Ltd', 'Afro International Foreign Exchange Bureau Limited', 'Aftab Currency Exchange Limited', 'Airtel UG', 'Akbar Money', 'Al Ansari Exchange', 
'Alawneh Exchange', 'Aldar', 'AL FARDAN EXCHANGE LLC', 'AL JAZEERA EXCHANGE', 'Al Malik AlDahery Exchange', 'AL Mulla Exchange', 'Al Nada International Exchange', 'Al Noaman Exchange Company',
 'Alpha Fortress Ltd', 'Al Razouki International Exchange Co LLC', 'Al-Taif Islamic Bank for Investment and Finance', 'Al Zaman Exchange WLL', 'ARY Exchange Company WLL', 'ASAP ALR Test Send Partner',
'ASAP Test Send Partner', 'Astero Pay PTY Limited', 'AyoRemit', 'Azimo', 'Bahrain Exchange Company WLL- Kuwait', 'Bahrain India International Exchange Co.', 'Bananapay LLC', 'Banco Delbank S.A',
'Bancore','Barraq Finance Company', 'Batelco Financial Services', 'Batelco Financial Services LTD', 'BEDSOPIA FZCO', 'Belmoney', 'Best Finance', 'BFC Bahrain', 'BICIG', 'Bin Yaala Exchange', 
'BlaBla Connect', 'BNB Senegal','BoB Finance SAL', 'Bolt Operations', 'Boss Revolution', 'BTC', 'Buckzy', 'ByteDance', 'ByteDance HK', 'Cadorim SA', 'CashMinute', 'CITY EXPRESS MONEY TRANSFER JAPAN CO LTD', 
'CLEVERDO PAYMENT TECHNOLOGIES LTD.','CLUB SUEDAMERIKA INTERNATIONAL GMBH', 'CN Travel - ASAP', 'Cornerstone Payment Solutions Ltd', 'CORPORATE ALLIANCE PAYMENTS PTY LTD', 'Cross ENF Inc', 'CrossPay', 
'Currenxie Limited', 'DADANADA NL SARL', 'DANESH Exchange','DAYTONA', 'Dee_Money', 'Digipay', 'DigiPay', 'Digipay CD', 'Digipay CG', 'Digital International Bank', 'DigitWace LLC', 'Dimagi Inc', 'dLocal',
'Doha Exchange Co', 'E4F', 'Easy Financial Services', 'Ebix Cash', 'Edens Capital Inc','e Kwacha Global Limited', 'Elephant Money Ltd', 'Elevate Pay INC', 'Equity Bank South Sudan Limited', 'EZ Money',
'Family Link Money Transfer', 'Financial House Limited', 'FINANCIERA GOLDEN LEAF S.A.', 'FinShot. Inc', 'Flex Money South Sudan', 'Flocash Limited', 'FOMO Pay PTE LTD - Fomopay', 'Fynte', 
'GCC Exchange UK Limited', 'GEA Limited', 'GIVEDIRECTLY INC', 'GLOBAL REMIT CURRENCY SERVICES LTD', 'GME', 'GME_Korea', 'GMoney', 'Golden Money Transfer', 'Golden Money Transfer Inc - Deal Based',
'Guavapay Limited', 'Gulf Overseas Exchange Co. LLC', 'Habib Qatar Intl Exchange', 'HAMI', 'Hanpass Co. Ltd', 'HHFSL', 'Home Remitt Pty', 'IC', 'iFAST Global Bank', 'Ileero', 'Ileero_ALR', 
'Intercambio Express INC', 'INTERCASH', 'International Payout Systems Canada INC','INTERNATIONAL PAYOUT SYSTEMS CANADA INC - same pf.py ccy', 'ISEND', 'Istanbul Odeme ve Elektronil Para A.S', 
'I-Transfer Money Movers SA', 'Japan Remittance Finance co.ltd.', 'Kaah Express F. S. Inc', 'KCB', 'Klarpay AG', 'KMB Money', 'Kuwait Bahrain International Exchange Company', 
'Kuwait Bahrain International Exchange Company 1', 'Kuwait Bahrain International Exchange Company deal', 'Kuwait India International Exchange Co. WLL', 'Kwattel', 'La-Cederi Bureau De Change',
'Lemonade', 'Lemonade Technology Limited - Deal Based', 'Lulu Exchange AE', 'Lulu Exchange Company WLL Deal', 'Lulu Exchange Company WLL- Kuwait', 'Lulu Exchange OM', 
'Lulu Money Singapore PTE LTD', 'MAGNAJAY MONEY REMITTANCE', 'Mama_Money', 'MasterCard', 'Master Remit', 'Maxitransfers LLC', 'Merchant_Trade', 'MOBEX', 'Modern Exchange co LLC',
'Money Ex', 'Money Gram Payment systems', 'Moneygram South Africa', 'MONEYMATCH SDN BHD', 'MoneyNet', 'Moneytrans', 'MONISNAP', 'Monty', 'Monyglob', 'More Money Transfers S.A.',
'MTN South Sudan', 'MTN South Sudan Test','MTN UG', 'Muhammad Musa Abu Zaid and Partner Exchange Compan', 'Nairagram - Send', 'Nala Payments Canada Inc', 'NationalEx', 'NEC', 
'NEC Remit', 'Neema Services', 'Neutronpay Inc', 'NFE', 'Nilos Financial Services LTD', 'NobelFin', 'Nuvei Limited', 'Omnex', 'Omnex INC - DEAL BASED', 
'Optiswift Technologies Limited', 'Orokii Inc', 'Panda Remit', 'Papara Elektronik', 'Paragram Odeme Hizmetleri A.S Turkey', 'Payhada Co.LTD', 'PayInc Group Limited', 
'Paypoint', 'Payporter', 'PAYSEND', 'PAYWISER_IMT', 'Paywiser Ltd', 'PEEXIT', 'Pesapeer Incorporated', 'Placid_Express', 'PNC Global Transfers Inc', 'Pre Pay Electronics Payments Ltd',
'Prime Bank Limited', 'Pronto', 'PT REYHAN PUTRA MANDIRI - Remitpro', 'Pula Advisors AG', 'PyWave Network Technology Limited', 'Queen Bee Capital', 'RAA Express', 'Raenest Inc',
'Rana Financial INC', 'Red Chapina Sociedad Anonima', 'Remitly Inc', 'Remitly inc - Deal Based', 'REMITTANCES HUB ENTERPRISES CANADA LTD.', 'RIA', 'Rightqode INC', 
'RocketRemit', 'Roxe', 'Safaricom', 'SafariRemit', 'SAVE MONEY TRANSFER INC', 'SBI Cosmoney Co. Ltd.', 'Sefremit Pty Ltd', 'Seif Money', 'Sender Odeme Hizmetleri AS',
'SendMN', 'Send Money India', 'Send Wave', 'SentBe', 'Shanawani Exchange Company', 'SHIFT','Sibilla Solutions Limited', 'Sigue', 'Silent Roar Media', 
'SIMBA MONEY LIMITED', 'SimbaPay', 'SingX.co', 'SMALL WORLD', 'Speedy', 'SPOTON', 'Sterling Exchange', 'SUNWAY', 'SUPERSONICZ Ltd', 'Suvi Inc', 'Suyool SAL', 
'Swiss Exchange LLC', 'Syion Private Limited', 'TaptapSend', 'TECHWEST PTY LTD', 'Terra Payment Services Italy', 'Titanes Money Transfer', 'TP Pilot', 'TP TZ',
'TPTZ Airtel KE Onnet', 'Transamericas Services Corporation', 'TransFast', 'Transferwise', 'Triple A Technologies Pte Ltd', 'Twiga Technology LLC', 'Tylersoft Technologies',
'UAE EXCHANGE CENTRE W L L KUWAIT', 'UAE EXCHANGE CENTRE W L L KUWAITdeal', 'Ultra Group', 'UNIMONI EXCHANGE LLC','UNIMONI OMAN', 'Uniteller', 'UPESI', 'UPT', 
'Uremit', 'URemit Money Transfer B2B', 'Venture Garden Nigeria Ltd', 'Viamericas Corporation', 'Visa Payments Ltd', 'VK CONSULTANCY PTY LIMITED', 'Vodacom TZ', 
'WesternUnion', 'WingMoney', 'WireBarley', 'WireBarley Time Based', 'Wise', 'Wise_Biz', 'WorldRemit', 'WSL Global Limited', 'XTransfer Limited', 'YEEPAYHK',
'Zam Zam Money Exchange LLC Co', 'ZeePay', 'ZIYAD MUBAYRIK ALRADDADI TRADING ESTABLISHMENT']  # Example source partner
    # Example source partners
   
     
    # Create an empty DataFrame to store all results
    all_results = pd.DataFrame()  # Initialize an empty DataFrame for final results
    
    # Iterate through each source partner to filter and display results
    for partner in source_partners:
        print(f"\nProcessing for {partner}...")  # Indicate which partner is being processed
        
        # Filter and display top rows for the current partner
        top_rows = filter_and_display(filtered_df2, partner)  # Get top rows for the current partner
        print(f"Top rows for {partner}:\n", top_rows)  # Display the top rows
        
        # Compare and filter top rows
        top_rows_filtered = compare_and_filter(top_rows, filtered_df1)  # Filter top rows based on filtered_df1
        print(f"Filtered top rows for {partner}:\n", top_rows_filtered)  # Display the filtered top rows
        
        # Merge duplicate rows
        final_results = merge_duplicate_rows(top_rows_filtered)  # Merge any duplicate rows in the filtered results
        
        # Load the R_D.csv file path
        r_d_file_path = 'R_D.csv'  # Path to the R_D.csv file
        
        # Add the regional director column
        merged_results_with_director = add_regional_director(final_results, r_d_file_path)  # Add regional director info
        
        # Add Partner_Gsv column based on the original DataFrame
        merged_results_with_partner_gsv = add_partner_gsv(merged_results_with_director, df)  # Add Partner_Gsv
        
        # Append the results to the all_results DataFrame
        all_results = pd.concat([all_results, merged_results_with_partner_gsv], ignore_index=True)  # Combine results
        
        # Display the updated results
        print(f"Updated Merged Results with Regional Director and Partner Gsv for {partner}:\n", merged_results_with_partner_gsv)  # Show updated results

    # Rename 'gsv' to 'TerraPay_gsv' in the final results
    final_results_with_renamed_gsv = rename_gsv_column(all_results)  # Rename 'gsv' to 'TerraPay_gsv'

    # Save the updated final results to a CSV file
    output_file_with_renamed_gsv = 'Partner_Opportunities.xlsx'  # Define the output file name
    final_results_with_renamed_gsv.to_excel(output_file_with_renamed_gsv, index=False)  # Save the DataFrame to a CSV file
    print(f"\nFinal results with 'TerraPay_gsv' have been saved to {output_file_with_renamed_gsv}.")


                                  source_partner  source_country destination_country      gsv
3     33 Remittance Company Limited - smartswift        Thailand               India  85967.0
4     33 Remittance Company Limited - smartswift        Thailand           Indonesia  24894.0
7     33 Remittance Company Limited - smartswift        Thailand            Malaysia  15450.0
10    33 Remittance Company Limited - smartswift        Thailand              Turkey  14314.0
8     33 Remittance Company Limited - smartswift        Thailand            Pakistan  14149.0
...                                          ...             ...                 ...      ...
2275                           iFAST Global Bank  United Kingdom              Turkey     10.0
2255                           iFAST Global Bank  United Kingdom             Estonia      3.0
2253                           iFAST Global Bank  United Kingdom             Croatia      1.0
2257                           iFAST Global Bank  United Kin

C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_partner'] = source_partner  # Correct assignment for the current partner
C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_partner'] = source_partner  # Correct assignment for the current partner
C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarn

Top rows for 9PAY JOINT STOCK COMPANY:
      source_country destination_country      gsv            source_partner
0     Åland Islands            Viet Nam  85001.0  9PAY JOINT STOCK COMPANY
1     Åland Islands               Kenya  15868.0  9PAY JOINT STOCK COMPANY
5091       Zimbabwe               India  22701.0  9PAY JOINT STOCK COMPANY
5092       Zimbabwe        South Africa  17959.0  9PAY JOINT STOCK COMPANY
5093       Zimbabwe         Netherlands  13910.0  9PAY JOINT STOCK COMPANY
...             ...                 ...      ...                       ...
12          Albania            Viet Nam  40428.0  9PAY JOINT STOCK COMPANY
13          Albania               Kenya  24127.0  9PAY JOINT STOCK COMPANY
14          Albania              France  22272.0  9PAY JOINT STOCK COMPANY
15          Albania               Spain  20066.0  9PAY JOINT STOCK COMPANY
16          Albania            Portugal  15137.0  9PAY JOINT STOCK COMPANY

[1193 rows x 4 columns]
Filtered top rows for 9PAY JOINT ST

C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_partner'] = source_partner  # Correct assignment for the current partner
C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_partner'] = source_partner  # Correct assignment for the current partner
C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarn

Updated Merged Results with Regional Director and Partner Gsv for Afex transferencias y cambios Ltda:
                        source_partner source_country destination_country        gsv Partner_Gsv existing/new regional_director
0  Afex transferencias y cambios Ltda          Chile             Belgium   483562.0        <NA>          NEW            Philip
1  Afex transferencias y cambios Ltda          Chile            Colombia  1541021.0   1662752.0     EXISTING            Philip
2  Afex transferencias y cambios Ltda          Chile              France   811178.0        <NA>          NEW            Philip
3  Afex transferencias y cambios Ltda          Chile             Germany  1148057.0        <NA>          NEW            Philip
4  Afex transferencias y cambios Ltda          Chile               India   335217.0        <NA>          NEW            Philip
5  Afex transferencias y cambios Ltda          Chile               Italy   440334.0        <NA>          NEW            Philip
6  Afex 

...

Updated Merged Results with Regional Director and Partner Gsv for WorldRemit:
     source_partner            source_country destination_country           gsv  Partner_Gsv existing/new regional_director
0       WorldRemit                 Australia          Bangladesh    40785924.0     664316.0     EXISTING           Praveen
1       WorldRemit                 Australia             Germany     9418964.0         <NA>          NEW           Praveen
2       WorldRemit                 Australia               Ghana    14536937.0         <NA>          NEW           Praveen
3       WorldRemit                 Australia               India  1271119070.0  777998305.0     EXISTING           Praveen
4       WorldRemit                 Australia               Kenya   100661663.0         <NA>          NEW           Praveen
..             ...                       ...                 ...           ...          ...          ...               ...
396     WorldRemit  United States of America             Lib

C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_partner'] = source_partner  # Correct assignment for the current partner
C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_partner'] = source_partner  # Correct assignment for the current partner
C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarn

Top rows for YEEPAYHK:
      source_country destination_country      gsv source_partner
0     Åland Islands            Viet Nam  85001.0       YEEPAYHK
1     Åland Islands               Kenya  15868.0       YEEPAYHK
5091       Zimbabwe               India  22701.0       YEEPAYHK
5092       Zimbabwe        South Africa  17959.0       YEEPAYHK
5093       Zimbabwe         Netherlands  13910.0       YEEPAYHK
...             ...                 ...      ...            ...
12          Albania            Viet Nam  40428.0       YEEPAYHK
13          Albania               Kenya  24127.0       YEEPAYHK
14          Albania              France  22272.0       YEEPAYHK
15          Albania               Spain  20066.0       YEEPAYHK
16          Albania            Portugal  15137.0       YEEPAYHK

[1193 rows x 4 columns]
Filtered top rows for YEEPAYHK:
      source_country destination_country           gsv source_partner
1736      Hong Kong         Philippines  1722181320.0       YEEPAYHK
1737      Ho

C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_partner'] = source_partner  # Correct assignment for the current partner
C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_partner'] = source_partner  # Correct assignment for the current partner
C:\Users\SreekantPatnaik\AppData\Local\Temp\ipykernel_15564\817701400.py:38: SettingWithCopyWarn

Updated Merged Results with Regional Director and Partner Gsv for ZIYAD MUBAYRIK ALRADDADI TRADING ESTABLISHMENT:
                                    source_partner source_country destination_country           gsv Partner_Gsv existing/new regional_director
0  ZIYAD MUBAYRIK ALRADDADI TRADING ESTABLISHMENT   Saudi Arabia          Bangladesh   505853212.0        <NA>          NEW            Bassem
1  ZIYAD MUBAYRIK ALRADDADI TRADING ESTABLISHMENT   Saudi Arabia               Egypt    73553370.0        <NA>          NEW            Bassem
2  ZIYAD MUBAYRIK ALRADDADI TRADING ESTABLISHMENT   Saudi Arabia               Ghana    26200926.0        <NA>          NEW            Bassem
3  ZIYAD MUBAYRIK ALRADDADI TRADING ESTABLISHMENT   Saudi Arabia               India  2478180376.0        <NA>          NEW            Bassem
4  ZIYAD MUBAYRIK ALRADDADI TRADING ESTABLISHMENT   Saudi Arabia               Kenya    26022010.0        <NA>          NEW            Bassem
5  ZIYAD MUBAYRIK ALRADDADI TRADI